# Segmenting and Clustering Neighborhoods in Toronto

## Ben Haworth, 23rd May 2020

### Instructions for assignment

In this assignment, you will be required to explore, segment, and cluster the neighborhoods in the city of Toronto. However, unlike New York, the neighborhood data is not readily available on the internet. What is interesting about the field of data science is that each project can be challenging in its unique way, so you need to learn to be agile and refine the skill to learn new libraries and tools quickly depending on the project.

For the Toronto neighborhood data, a Wikipedia page exists that has all the information we need to explore and cluster the neighborhoods in Toronto. You will be required to scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas dataframe so that it is in a structured format like the New York dataset.

Once the data is in a structured format, you can replicate the analysis that we did to the New York City dataset to explore and cluster the neighborhoods in the city of Toronto.

Your submission will be a link to your Jupyter Notebook on your Github repository.

### Guidelines to structure and formatting of this notebook

Each section has been numbered according to the three parts of the assignment. I have decided to create one notebook only.

Instructions in the assignment which should result in a specific output are in bold italics, as below.

##### Instructions in the assignment in bold italics

My own notes are regular text, as below.

Regular text

## 0. Import generic modules

In [155]:
# import modules needed for typical data analysis steps

import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## 1. Scrape Neighbourhood data from Wikipedia

##### Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe.

##### The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [156]:
dfNeighborhoods = pd.DataFrame(columns=['PostalCode', 'Borough', 'Neighborhood'])

Data scraped using Beautiful Soup having looked at several posts:

* Using instructions from https://simpleanalytical.com/how-to-web-scrape-wikipedia-python-urllib-beautiful-soup-pandas
* See also instructions from https://stackoverflow.com/questions/50355577/scraping-wikipedia-tables-with-python-selectively
* See also instructions from https://medium.com/analytics-vidhya/web-scraping-wiki-tables-using-beautifulsoup-and-python-6b9ea26d8722

In [157]:
# import modules for web scraping and formatting

import requests
from bs4 import BeautifulSoup

In [158]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

request = requests.get(url).text
soup = BeautifulSoup(request,'lxml')

myTable = soup.find('table',{'class':'wikitable sortable'})

postalCode=[]
borough=[]
neighborhood=[]

for row in myTable.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        postalCode.append(cells[0].find(text=True))
        borough.append(cells[1].find(text=True))
        neighborhood.append(cells[2].find(text=True))
        
dfNeighborhoods['PostalCode'] = postalCode
dfNeighborhoods['Borough'] = borough
dfNeighborhoods['Neighborhood'] = neighborhood

dfNeighborhoods = dfNeighborhoods.replace(r'\n','', regex=True) # remove the \n new line characters

dfNeighborhoods.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


##### Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

Removed the Not assigned boroughs using simple slicing

In [159]:
dfNeighborhoods = dfNeighborhoods[dfNeighborhoods['Borough'] != 'Not assigned']

dfNeighborhoods.reset_index(drop=True, inplace=True)

dfNeighborhoods.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


##### More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

This seemed to happen without any further action required. See for example:

In [160]:
dfNeighborhoods[dfNeighborhoods['PostalCode']=='M5A']

,PostalCode,Borough,Neighborhood
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"


##### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

There aren't any rows where the neighborhood is Not assigned

In [161]:
dfNeighborhoods[dfNeighborhoods['Neighborhood']=='Not assigned']

,PostalCode,Borough,Neighborhood


However, if there were Not assigned neighborhoods, then the following should take care of it:

In [162]:
dfNeighborhoods['Neighborhood'].loc[(dfNeighborhoods['Neighborhood'] == 'Not assigned')] = dfNeighborhoods['Borough']

dfNeighborhoods.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Proof that the replacement would work can be seen in the copied data frame below, where I add a Not assigned neighborhood, and the replace the value with the Borough

In [163]:
dfNeighborhoodsCopy = dfNeighborhoods

row = pd.DataFrame([['M99','Test Borough', 'Not assigned']], columns=['PostalCode', 'Borough', 'Neighborhood'])

dfNeighborhoodsCopy = dfNeighborhoodsCopy.append(row, ignore_index=True)

dfNeighborhoodsCopy.tail(1)

,PostalCode,Borough,Neighborhood
103,M99,Test Borough,Not assigned


In [164]:
dfNeighborhoodsCopy['Neighborhood'].loc[(dfNeighborhoodsCopy['Neighborhood'] == 'Not assigned')] = dfNeighborhoodsCopy['Borough']

dfNeighborhoodsCopy.tail(1)

,PostalCode,Borough,Neighborhood
103,M99,Test Borough,Test Borough


##### In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [165]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(dfNeighborhoods['Borough'].unique()),
        dfNeighborhoods.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


## 2.  Get the coordinates of each neighborhood

Attempting to use the geocoder example from instructions.

In [166]:
#import modules for geocoding

import geocoder # import geocoder

In [167]:
# initialize your variable to None
lat_lng_coords = None

postal_code = 'M5G'

# loop until you get the coordinates
#while(lat_lng_coords is None):
#  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
#  lat_lng_coords = g.latlng

#latitude = lat_lng_coords[0]
#longitude = lat_lng_coords[1]

Tried this and it just hangs. Commented out so that I don't accidentally run it. I decided to import the CSV file instead.

Import CSV file to dataframe.

In [168]:
geoData = pd.read_csv('http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv')
geoData.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
geoData.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


##### Use the Geocoder package or the csv file to create the following dataframe

Merge neighborhoods dataframe with coordinates.

N.B. This data frame does not have the same sort order as the one in the screenshot. The sort order for the screenshot is unclear.

In [169]:
dfNeighborhoodsCoords = dfNeighborhoods.merge(geoData, how='inner', on='PostalCode')
dfNeighborhoodsCoords.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


## 3. Explore and cluster the neighborhoods in Toronto

Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.

Just make sure:

1. to add enough Markdown cells to explain what you decided to do and to report any observations you make.
2. to generate maps to visualize your neighborhoods and how they cluster together.

In [170]:
# import k-means for clustering
from sklearn.cluster import KMeans

# import modules for mapping
import folium # map rendering library
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

Use geopy library to get the latitude and longitude values of Toronto.

In [171]:
# code lifted from the New York example and amended

address = 'Toronto, ON'

geolocator = Nominatim(user_agent="ben_haworth")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto, ON, Canada are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto, ON, Canada are 43.6534817, -79.3839347.


### 3.1. Create a map of Toronto with neighborhoods superimposed on top.

In [172]:
# code lifted from the New York example and amended

# create map of Toronto using latitude and longitude values
mapToronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(dfNeighborhoodsCoords['Latitude']
                                           , dfNeighborhoodsCoords['Longitude']
                                           , dfNeighborhoodsCoords['Borough']
                                           , dfNeighborhoodsCoords['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(mapToronto)  
    
mapToronto

The instructions suggest limiting to boroughs with Toronto in the name. The resulting data frame has a similar number of neighbourhoods to Manhattan (39 Toronto vs 40 Manhattan).

In [173]:
# code lifted from the New York example and amended

dfNeighborhoodsToronto = dfNeighborhoodsCoords[dfNeighborhoodsCoords['Borough'].str.contains('Toronto')]
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(dfNeighborhoodsToronto['Borough'].unique()),
        dfNeighborhoodsToronto.shape[0]
    )
)

The dataframe has 4 boroughs and 39 neighborhoods.


In [174]:
# code lifted from the New York example and amended

# create map of Toronto using latitude and longitude values
mapTorontoSliced = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(dfNeighborhoodsToronto['Latitude']
                                           , dfNeighborhoodsToronto['Longitude']
                                           , dfNeighborhoodsToronto['Borough']
                                           , dfNeighborhoodsToronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(mapTorontoSliced)  
    
mapTorontoSliced

### 3.2 Explore neighborhoods in Toronto

Define Foursquare Credentials and Version

In [175]:
# @hidden_cell
CLIENT_ID = 'QE1B143D1BIW2SV4WM1PUYS3253A120VWMCZCRH3OFJ0RTMP' # your Foursquare ID
CLIENT_SECRET = '4NT14KJ0EV4BCL2PJRP0STZZ2IQRBHJIRTVLZKK3E10YVKBT' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

Function to iterate over all neighborhoods in data frame

In [176]:
# code lifted from the New York example and amended

def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [177]:
# get venues for dfNeighborhoodsToronto
dfTorontoVenues = getNearbyVenues(names=dfNeighborhoodsToronto['Neighborhood'],
                                   latitudes=dfNeighborhoodsToronto['Latitude'],
                                   longitudes=dfNeighborhoodsToronto['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town,

Check the size of the resulting dataframe.

In [178]:
print(dfTorontoVenues.shape)
dfTorontoVenues.head()

(1613, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
3,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
4,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


Check how many venues were returned for each neighborhood.

In [179]:
dfTorontoVenues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,54,54,54,54,54,54
"Brockton, Parkdale Village, Exhibition Place",23,23,23,23,23,23
Business reply mail Processing Centre,17,17,17,17,17,17
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",15,15,15,15,15,15
Central Bay Street,65,65,65,65,65,65
Christie,17,17,17,17,17,17
Church and Wellesley,77,77,77,77,77,77
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,38,38,38,38,38,38


Double check all the neighborhoods are still in the dataframe.

In [180]:
dfTorontoVenues['Neighborhood'].nunique()

39

In [181]:
dfTorontoVenues['Venue Category'].sort_values().unique()

array(['Afghan Restaurant', 'Airport', 'Airport Food Court',
       'Airport Gate', 'Airport Lounge', 'Airport Service',
       'Airport Terminal', 'American Restaurant', 'Antique Shop',
       'Aquarium', 'Art Gallery', 'Art Museum', 'Arts & Crafts Store',
       'Asian Restaurant', 'Athletics & Sports', 'Auto Workshop',
       'BBQ Joint', 'Baby Store', 'Bagel Shop', 'Bakery', 'Bank', 'Bar',
       'Baseball Stadium', 'Basketball Stadium', 'Beach',
       'Bed & Breakfast', 'Beer Bar', 'Beer Store', 'Belgian Restaurant',
       'Bistro', 'Boat or Ferry', 'Bookstore', 'Boutique',
       'Brazilian Restaurant', 'Breakfast Spot', 'Brewery',
       'Bubble Tea Shop', 'Building', 'Burger Joint', 'Burrito Place',
       'Bus Line', 'Business Service', 'Butcher', 'Café',
       'Cajun / Creole Restaurant', 'Camera Store', 'Candy Store',
       'Caribbean Restaurant', 'Cheese Shop', 'Chinese Restaurant',
       'Chocolate Shop', 'Church', 'Climbing Gym', 'Clothing Store',
       'Cocktail Ba

### 3.3 Analyse each neighborhood

Onehot encoding of venue categories.

An interesting problem was found when trying to carry out one hot encoding and then adding back in the Neighborhood name as a column. It appears one of the venue categories is also called "Neighbourhood".

In [182]:
dfTorontoVenues[dfTorontoVenues['Venue Category'] == 'Neighborhood']

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
262,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
408,"Richmond, Adelaide, King",43.650571,-79.384568,Downtown Toronto,43.653232,-79.385296,Neighborhood
508,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752,Harbourfront,43.639526,-79.380688,Neighborhood
943,Studio District,43.659526,-79.340923,Leslieville,43.662070,-79.337856,Neighborhood


In order to avoid the conflict, I am dropping the Venue Category called "Neighborhood" before carrying out the one hot encoding.

In [183]:
dfTorontoVenues = dfTorontoVenues[dfTorontoVenues['Venue Category'] != 'Neighborhood']

Double check that they have definitely been removed.

In [184]:
dfTorontoVenues[dfTorontoVenues['Venue Category'] == 'Neighborhood']

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category


In [185]:
# code lifted from the New York example and amended. 

# one hot encoding
dfTorontoOnehot = pd.get_dummies(dfTorontoVenues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
dfTorontoOnehot['Neighborhood'] = dfTorontoVenues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [dfTorontoOnehot.columns[-1]] + list(dfTorontoOnehot.columns[:-1])
dfTorontoOnehot = dfTorontoOnehot[fixed_columns]

dfTorontoOnehot.head()

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

Examine the new dataframe size.

In [186]:
dfTorontoOnehot.shape

(1609, 239)

Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category.

In [187]:
dfTorontoGrouped = dfTorontoOnehot.groupby('Neighborhood').mean().reset_index()
dfTorontoGrouped

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.018519,0.000000,0.000000,0.000000,0.000000,0.000000,0.018519,0.000000,0.018519,0.037037,0.000000,0.000000,0.000000,0.018519,0.018519,0.000000,0.037037,0.000000,0.000000,0.018519,0.000000,0.000000,0.000000,0.000000,0.018519,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.037037,0.000000,0.000000,0.000000,0.000000,0.037037,0.000000,0.000000,0.000000,0.000000,0.018519,0.055556,0.074074,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.018519,0.000000,0.018519,

Confirm the new size.

In [188]:
dfTorontoGrouped.shape

(39, 239)

Print each neighborhood along with the top 5 most common venues.

In [189]:
# code lifted from the New York example and amended

num_top_venues = 5

for hood in dfTorontoGrouped['Neighborhood']:
    print("----"+hood+"----")
    temp = dfTorontoGrouped[dfTorontoGrouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.07
1        Cocktail Bar  0.06
2            Beer Bar  0.04
3  Seafood Restaurant  0.04
4              Bakery  0.04


----Brockton, Parkdale Village, Exhibition Place----
            venue  freq
0            Café  0.13
1  Breakfast Spot  0.09
2       Nightclub  0.09
3     Coffee Shop  0.09
4          Bakery  0.04


----Business reply mail Processing Centre----
                  venue  freq
0  Gym / Fitness Center  0.06
1         Garden Center  0.06
2            Comic Shop  0.06
3                  Park  0.06
4      Recording Studio  0.06


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                 venue  freq
0      Airport Service  0.20
1     Airport Terminal  0.13
2  Rental Car Location  0.07
3   Airport Food Court  0.07
4         Airport Gate  0.07


----Central Bay Street----
                 venue  freq
0          Coffee Shop  0.17
1   Ita

Function to sort the venues in descending order.

In [190]:
# code lifted from the New York example and amended

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Create the new dataframe and display the top 10 venues for each neighborhood.

In [191]:
# code lifted from the New York example and amended

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = dfTorontoGrouped['Neighborhood']

for ind in np.arange(dfTorontoGrouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(dfTorontoGrouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Restaurant,Beer Bar,Bakery,Seafood Restaurant,Cheese Shop,Café,Fish Market,Hotel
1,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Nightclub,Coffee Shop,Climbing Gym,Burrito Place,Restaurant,Italian Restaurant,Stadium,Intersection
2,Business reply mail Processing Centre,Pizza Place,Auto Workshop,Comic Shop,Recording Studio,Restaurant,Butcher,Burrito Place,Brewery,Skate Park,Smoke Shop
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Terminal,Plane,Rental Car Location,Boat or Ferry,Harbor / Marina,Sculpture Garden,Boutique,Airport Lounge,Airport Gate
4,Central Bay Street,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Japanese Restaurant,Salad Place,Bubble Tea Shop,Ice Cream Shop,Burger Joint,Bar


### 3.4 Cluster Neighborhoods

Run k-means to cluster the neighborhood into 5 clusters.

In [192]:
# code lifted from the New York example and amended


# set number of clusters
kclusters = 5

dfTorontoGroupedClustering = dfTorontoGrouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dfTorontoGroupedClustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [193]:
# code lifted from the New York example and amended

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

dfTorontoMerged = dfNeighborhoodsToronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
dfTorontoMerged = dfTorontoMerged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

In [194]:
dfTorontoMerged # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Park,Bakery,Pub,Café,Breakfast Spot,Theater,Yoga Studio,Spa,Ice Cream Shop
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Sushi Restaurant,College Cafeteria,Yoga Studio,Distribution Center,Park,Music Venue,Mexican Restaurant,Japanese Restaurant,Italian Restaurant
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Clothing Store,Coffee Shop,Middle Eastern Restaurant,Japanese Restaurant,Italian Restaurant,Cosmetics Shop,Café,Bubble Tea Shop,Restaurant,Bookstore
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Café,Cocktail Bar,Gastropub,American Restaurant,Beer Bar,Italian Restaurant,Lingerie Store,Department Store,Creperie
19,M4E,East Toronto,The Beaches,43.676357,-79.293031,3,Trail,Health Food Store,Pub,Dog Run,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Yoga Studio
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Cocktail Bar,Restaurant,Beer Bar,Bakery,Seafood Restaurant,Cheese Shop,Café,Fish Market,Hotel
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,0,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Japanese Restaurant,Salad Place,Bubble Tea Shop,Ice Cream Shop,Burger Joint,Bar
25,M6G,Downtown Toronto,Christie,43.669542,-79.422564,0,Grocery Store,Café,Park,Baby Store,Candy Store,Italian Restaurant,Athletics & Sports,Diner,Nightclub,Restaurant
30,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568,0,Coffee Shop,Café,Restaurant,Gym,Hotel,Clothing Store,Thai Restaurant,Deli / Bodega,Steakhouse,Sushi Restaurant
31,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259,0,Bakery,Pharmacy,Music Venue,Brewery,Portuguese Restaurant,Pool,Middle Eastern Restaurant,Art Gallery,Bar,Supermarket


Visualize the resulting clusters

In [195]:
# code lifted from the New York example and amended

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dfTorontoMerged['Latitude']
                                  , dfTorontoMerged['Longitude']
                                  , dfTorontoMerged['Neighborhood']
                                  , dfTorontoMerged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

The result seems to suggest that Toronto is quite homogenous, with Cluster 0 (shown in red) being the most common by far. 

### All done!